<a href="https://colab.research.google.com/github/LukasRimkus/100-Prisoners-Riddle/blob/main/NLP_CW2_TASK1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Natural Language Processing Coursework 2 - Task 1

## Open Google Colab

In [222]:
from google.colab import drive

colab_path = '/content/drive'
drive.mount(colab_path)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


##Importing Essential Libraries

Import libraries used in the coursework

In [223]:
import os
import nltk
import string
import re

from nltk.stem import PorterStemmer, WordNetLemmatizer
from nltk.tokenize import word_tokenize, sent_tokenize
from nltk.cluster import euclidean_distance, cosine_distance

from collections import Counter
import time
import itertools
import gensim
from gensim.models import Word2Vec, FastText
import numpy as np
from collections import Counter
import random
import copy
import time

from sklearn.cluster import KMeans, AgglomerativeClustering, Birch
from sklearn.model_selection import train_test_split 

Download required sets and set the constants. 

In [224]:
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('omw-1.4')

BASE_PATH = "/content/drive/MyDrive/product_reviews"

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


##Contraction Map Definition

The defined dictionary is used for expanding contractions (like **can't** is replaced by **cannot**) inside the documents.

In [225]:
#@title Contractions Map 
"""
This snippet of code was taken from:
https://github.com/kootenpv/contractions/blob/master/contractions/data/contractions_dict.json

As the library could not be downloaded to do it, so I found handy to make use of it
to remove unnecessary bits of code quite easily with minimal invested resources. 
"""

CONTRACTIONS = {
    "I'm": "I am",
    "I'm'a": "I am about to",
    "I'm'o": "I am going to",
    "I've": "I have",
    "I'll": "I will",
    "I'll've": "I will have",
    "I'd": "I would",
    "I'd've": "I would have",
    "Whatcha": "What are you",
    "amn't": "am not",
    "ain't": "are not",
    "aren't": "are not",
    "'cause": "because",
    "can't": "cannot",
    "can't've": "cannot have",
    "could've": "could have",
    "couldn't": "could not",
    "couldn't've": "could not have",
    "daren't": "dare not",
    "daresn't": "dare not",
    "dasn't": "dare not",
    "didn't": "did not",
    "didn’t": "did not",
    "don't": "do not",
    "don’t": "do not",
    "doesn't": "does not",
    "e'er": "ever",
    "everyone's": "everyone is",
    "finna": "fixing to",
    "gimme": "give me",
    "gon't": "go not",
    "gonna": "going to",
    "gotta": "got to",
    "hadn't": "had not",
    "hadn't've": "had not have",
    "hasn't": "has not",
    "haven't": "have not",
    "he've": "he have",
    "he's": "he is",
    "he'll": "he will",
    "he'll've": "he will have",
    "he'd": "he would",
    "he'd've": "he would have",
    "here's": "here is",
    "how're": "how are",
    "how'd": "how did",
    "how'd'y": "how do you",
    "how's": "how is",
    "how'll": "how will",
    "isn't": "is not",
    "it's": "it is",
    "'tis": "it is",
    "'twas": "it was",
    "it'll": "it will",
    "it'll've": "it will have",
    "it'd": "it would",
    "it'd've": "it would have",
    "kinda": "kind of",
    "let's": "let us",
    "luv": "love",
    "ma'am": "madam",
    "may've": "may have",
    "mayn't": "may not",
    "might've": "might have",
    "mightn't": "might not",
    "mightn't've": "might not have",
    "must've": "must have",
    "mustn't": "must not",
    "mustn't've": "must not have",
    "needn't": "need not",
    "needn't've": "need not have",
    "ne'er": "never",
    "o'": "of",
    "o'clock": "of the clock",
    "ol'": "old",
    "oughtn't": "ought not",
    "oughtn't've": "ought not have",
    "o'er": "over",
    "shan't": "shall not",
    "sha'n't": "shall not",
    "shalln't": "shall not",
    "shan't've": "shall not have",
    "she's": "she is",
    "she'll": "she will",
    "she'd": "she would",
    "she'd've": "she would have",
    "should've": "should have",
    "shouldn't": "should not",
    "shouldn't've": "should not have",
    "so've": "so have",
    "so's": "so is",
    "somebody's": "somebody is",
    "someone's": "someone is",
    "something's": "something is",
    "sux": "sucks",
    "that're": "that are",
    "that's": "that is",
    "that'll": "that will",
    "that'd": "that would",
    "that'd've": "that would have",
    "em": "them",
    "there're": "there are",
    "there's": "there is",
    "there'll": "there will",
    "there'd": "there would",
    "there'd've": "there would have",
    "these're": "these are",
    "they're": "they are",
    "they've": "they have",
    "they'll": "they will",
    "they'll've": "they will have",
    "they'd": "they would",
    "they'd've": "they would have",
    "this's": "this is",
    "this'll": "this will",
    "this'd": "this would",
    "those're": "those are",
    "to've": "to have",
    "wanna": "want to",
    "wasn't": "was not",
    "we're": "we are",
    "we've": "we have",
    "we'll": "we will",
    "we'll've": "we will have",
    "we'd": "we would",
    "we'd've": "we would have",
    "weren't": "were not",
    "what're": "what are",
    "what'd": "what did",
    "what've": "what have",
    "what's": "what is",
    "what'll": "what will",
    "what'll've": "what will have",
    "when've": "when have",
    "when's": "when is",
    "where're": "where are",
    "where'd": "where did",
    "where've": "where have",
    "where's": "where is",
    "which's": "which is",
    "who're": "who are",
    "who've": "who have",
    "who's": "who is",
    "who'll": "who will",
    "who'll've": "who will have",
    "who'd": "who would",
    "who'd've": "who would have",
    "why're": "why are",
    "why'd": "why did",
    "why've": "why have",
    "why's": "why is",
    "will've": "will have",
    "won't": "will not",
    "won't've": "will not have",
    "would've": "would have",
    "wouldn't": "would not",
    "wouldn't've": "would not have",
    "y'all": "you all",
    "y'all're": "you all are",
    "y'all've": "you all have",
    "y'all'd": "you all would",
    "y'all'd've": "you all would have",
    "you're": "you are",
    "you've": "you have",
    "you'll've": "you shall have",
    "you'll": "you will",
    "you'd": "you would",
    "you'd've": "you would have",
}

## Constants

In [226]:
NUMBER_OF_WORDS = 50

##Pre-processing Class Definition 

Define Pre-processing class for cleaning the document text, converting it to cleaned tokens and adding multiwords to the index. 

In [227]:
class PreProcessing:
    """
    This is the class used to pre-process documents and search terms. 
    I found this more readable and comfortable to move this functionality to a 
    separate class than to have everything in one place. This is a more typical 
    approach used in industry.
    """
    
    def lower_doc(self, doc: str) -> str:
        """
        This method takes a document as string and converts all characters to lowercase.
        Return a lowercased doocument.
        """
        doc = doc.lower()
        return doc

    def remove_non_ascii_characters(self, doc: str) -> str:
        """
        This method removes non-ASCII characters from the document text.
        It has a document as string parameter.
        Returns a document with removed non-ASCII characters which are replaced by a space.
        """
        # Without an added space, the words could be merged together to one, which is unwanted.
        doc = re.sub(r'[^\x00-\x7F]+',' ', doc)
        return doc
    
    def remove_annotations(self, document: str) -> str:
        """
        This method removes citations like "[1]" from the document text.

        It has a document as string parameter.
        Returns a document with removed citations.
        """
        document = re.sub(r"\[.{0,5}\]", " ", document)

        return document

    def remove_unnecessary_symbols(self, document: str) -> str:
        document = document.replace("##", " ")
        document = document.replace("\n", " ")
        return document

    def remove_punctuation(self, sents: list) -> list:
        """
        This method removes punctuation from the document text.

        It has a document as string parameter.
        Returns a document with removed punctuations.
        """

        # Remove an apostrophe from the list as they are removed later after removing contractions.
        punctuation = string.punctuation.replace("'", "")
        # punctuation = string.punctuation

        # It is actually faster to use a for loop for this than regex - 
        # https://datagy.io/python-remove-punctuation-from-string/. 
        # I do not use maketrans as I need to replace a punctuation by a space.
        for i, sent in enumerate(sents):
            for punct in punctuation:
                sent = sent.replace(punct, " ")
            
            sents[i] = sent

        return sents

    def remove_digits(self, sentences: list) -> list:
        """
        This method removes digits from the document text.

        It has a document as string parameter.
        Returns a document with removed digits.
        """
        for i, sent in enumerate(sentences):
            sentences[i] = re.sub("\d+", "", sent)

        return sentences

    def replace_contractions(self, sentences: list) -> list:
        """
        This method replaces contractions in the document text using a contraction map
        which is taken from a library on github repository. I found this a better 
        way to deal with contractions. The cost of this is small as there a lot of 
        contractions and the whole dictionary is quite small.

        It has a document as string parameter.
        Returns a document with replaced contraction equivalents.
        """
        
        # Split the document to tokens separated by a space.  
        for i, sent in enumerate(sentences):

            words = sent.split()
            filtered_words = list()
            
            for i, word in enumerate(words):
                if word in CONTRACTIONS:
                    filtered_words.extend(CONTRACTIONS[word].split())
                else:
                    filtered_words.append(word)
            
            # rejoin those tokens to a string
            sentences[i] = " ".join(filtered_words)
        
        return sentences

    def remove_apostrophes(self, sentences: list) -> list:
        """
        This method removes apostrophes from the document text. This is only done 
        after contractions expansion as ti should handle all the left cases with 
        unremoved apostrophes.

        It has a document as string parameter.
        Returns a document with removed apostrophes.
        """
        for i, sent in enumerate(sentences):
            sentences[i] = sent.replace("'", " ")

        return sentences

    def do_tokenisation(self, sents: list) -> list:
        tokens = [word_tokenize(sent) for sent in sents]

        return tokens

    def do_stemming(self, tokens_lists: list) -> list:
        """
        This method does stemming for each token.

        It has tokens as a list parameter.
        Returns a list of stemmed tokens.
        """

        stemmer = PorterStemmer()
        tokens = [[stemmer.stem(token) for token in tokens] for tokens in tokens_lists]
        return tokens

    def do_lemmatisation(self, tokens_lists: list) -> list:
        lemmatiser = WordNetLemmatizer()
        tokens = [[lemmatiser.lemmatize(token) for token in tokens] for tokens in tokens_lists]
        return tokens

    def remove_stopwords(self, tokens_lists: list) -> list:
        """
        This method removes words which do not give much value to index which are 
        known as stopwords. Also, words of size less than 2 characters are removed as well. 
        This method is used for cleaned document tokens with positions or only tokens for 
        cleaned search queries.

        It has tokens as a list parameter and a boolean value for determining if 
        the list contains only tokens or tuples of tokens and positions. 
        Returns a list of tokens with removed stopwords and tokens of size less than 
        2 characters.
        """

        # I may add my own stopwords
        stopwords = nltk.corpus.stopwords.words('english')
        extra_stopwords = ["was", "would", "could", "need", "has"]
        stopwords += extra_stopwords

        # Word "won" has a semantic meaning in the documents, especially when I expand 
        # "won't" to "will not", as the motivation to add "won" to stopwords list might have 
        # been to remove a stemmed form of "won't". But in my case, it removes a semantically
        # important word.
        tokens = [[token for token in tokens if token not in stopwords and len(token) > 1] for tokens in tokens_lists]
        
        return tokens

    def process_text(self, document: str) -> str:
        """
        This method makes use of methods created above for cleaning a text document.
        It does these cleaning steps:
        1) Making the text to the lower case.
        2) Removing redundant phrases.
        3) Removing non-ASCII characters.
        4) Replacing citations.
        5) Removing punctuation.
        6) Removing digits.
        7) Expanding contractions.
        8) Removing apostrophes.

        It has a unprocessed document string as its parameter. 
        Returns a cleaned document ready for tokenisation and stemming.
        """

        # make the case to lowercase for each word
        # may not need this 
        document = self.lower_doc(document)

        # remove non ascii characters from the document
        document = self.remove_non_ascii_characters(document)

        # remove annotations like '[+1]'
        document = self.remove_annotations(document)

        # remove some unnecessary symbols
        document = self.remove_unnecessary_symbols(document)

        # do sentence tokenisation here!
        sentences = sent_tokenize(document)

        # remove punctuation (remove apostrophes later after removing stopwords)
        sentences = self.remove_punctuation(sentences)

        # remove digits
        sentences = self.remove_digits(sentences)

        # replace contractions using a map which may not be detected by the tokenizer
        sentences = self.replace_contractions(sentences)

        # remove existing apostrophes after removing contractions
        sentences = self.remove_apostrophes(sentences)
        
        return sentences

    def process_document(self, document: str) -> list:
        """
        This method does the whole pre-processing for the document which becomes 
        ready for the building of the inverted index. 
        It does these steps:
        1) Cleaning the text like removing unnecessary text (look at 'process_text' method).
        2) Tokenisation of the cleaned string of text.
        3) Adding positions for tokens.
        4) Stemming tokens.
        5) Removing stopwords.
        6) Adding multiword terms from csv files.

        It has a string of a document and a list of csv multiword terms as its parameters. 
        Returns a list of tuples of terms and positions.
        """
        # prepare the document for the tokenisation
        sentences = self.process_text(document)

        # do tokenization of the cleaned text
        tokens_lists = self.do_tokenisation(sentences)

        # remove stopwords and single char tokens
        tokens_lists = self.remove_stopwords(tokens_lists)

        # do stemming of the tokens
        # tokens_lists = self.do_stemming(tokens_lists)
        tokens_lists = self.do_lemmatisation(tokens_lists)

        return tokens_lists


preprocessing = PreProcessing()

###Read Data

In [228]:
def read_data(path: str) -> list:
    """
    This method read files from a directory and then adds the data of each file
    into a list.

    It has a string of a path to read data from as its parameter. 
    Returns a list of tuples of a document text string and a tuple of the season 
    and episode integers.
    """

    def read_single_file(file):
        """
        This method read text for a given file.

        It has a string of a file name to read data from as its parameter. 
        Returns a title of the episode and the whole document text for pre-processing.
        Returns None if an exception was caught.
        """
        file_path = f"{path}/{file}"
        try:
            with open(file_path, 'r') as fd:
                text = fd.read()
                return text

        except IOError as e: 
            print(f"There was an IOError failure reading a text file at {file_path}. Error message: {e}")
        except Exception as e:
            print(f"There was some unrecognised failure reading a text file at {file_path}. Error message: {e}")
        

    # Access the directory given in path.
    all_file_names = os.listdir(path)
    text_file_names = list(filter(lambda x: x[-4:] == '.txt' and x != "README.txt", all_file_names))

    # Read the files and extract text and title for each document.
    documents = list(map(read_single_file, text_file_names)) 

    # remove .txt extension as it gives no information after finding those documents
    # text_file_names = [name[:-4].split(".") for name in text_file_names]
    
    return documents


def get_corpus_in_sentences():
    reviews = read_data(BASE_PATH)
    corpus = "\n".join(reviews)
    tokenised_sentence_corpus = preprocessing.process_document(corpus)

    tokenised_corpus = list()
    for sent in tokenised_sentence_corpus:
        tokenised_corpus.extend(sent)

    return tokenised_sentence_corpus, tokenised_corpus


In [229]:
def get_most_common_words(tokenised_corpus):
    all_word_dist = nltk.FreqDist(tokenised_corpus)
    most_common_words = [word for (word, _) in all_word_dist.most_common(NUMBER_OF_WORDS)]
    print(f"Most common words:\n{most_common_words}")

    most_common_words = np.array(most_common_words)
    tokenised_corpus = np.array(tokenised_corpus)

    return most_common_words

def get_reversed_most_common_words(most_common_words):
    reversed_words = [word[::-1] for word in most_common_words]
    print(f"\nMost common reversed words:\n{reversed_words}")
    return reversed_words

def get_all_words(most_common_words, reversed_words):
    all_words = list()
    for word, reversed_word in zip(most_common_words, reversed_words):
        all_words.append(word)
        all_words.append(reversed_word)
    return all_words

In [230]:
tokenised_sentence_corpus, tokenised_corpus = get_corpus_in_sentences()
most_common_words = get_most_common_words(tokenised_corpus)
reversed_words = get_reversed_most_common_words(most_common_words)
all_words = get_all_words(most_common_words, reversed_words)

Most common words:
['phone', 'use', 'one', 'ipod', 'router', 'player', 'camera', 'get', 'battery', 'diaper', 'work', 'product', 'great', 'like', 'feature', 'time', 'good', 'zen', 'problem', 'quality', 'also', 'sound', 'computer', 'software', 'really', 'micro', 'well', 'take', 'picture', 'thing', 'easy', 'even', 'used', 'creative', 'first', 'much', 'want', 'better', 'bag', 'mp', 'champ', 'look', 'size', 'go', 'music', 'norton', 'support', 'buy', 'price', 'little']

Most common reversed words:
['enohp', 'esu', 'eno', 'dopi', 'retuor', 'reyalp', 'aremac', 'teg', 'yrettab', 'repaid', 'krow', 'tcudorp', 'taerg', 'ekil', 'erutaef', 'emit', 'doog', 'nez', 'melborp', 'ytilauq', 'osla', 'dnuos', 'retupmoc', 'erawtfos', 'yllaer', 'orcim', 'llew', 'ekat', 'erutcip', 'gniht', 'ysae', 'neve', 'desu', 'evitaerc', 'tsrif', 'hcum', 'tnaw', 'retteb', 'gab', 'pm', 'pmahc', 'kool', 'ezis', 'og', 'cisum', 'notron', 'troppus', 'yub', 'ecirp', 'elttil']


The best way to do hyperparameter selection?

## Experiment

In [231]:
class DistributionalSemanticsExperiment:
    NUMBER_OF_WORDS = 50

    def __init__(self, random_trials, tokenised_sentence_corpus, all_words, most_common_words, reversed_words, N, d, window, min_count, iter, alpha):
        self.random_trials = random_trials
        self.tokenised_sentence_corpus = tokenised_sentence_corpus
        self.all_words = all_words
        self.most_common_words = most_common_words
        self.reversed_words = reversed_words
        self.N = N
        self.d = d
        self.window = window
        self.min_count = min_count
        self.iter = iter
        self.alpha = alpha

    def get_results(self):
        percentages = self.do_experiment_given_number_of_times()

        mean, std = percentages.mean(), percentages.std()

        return mean, std


    def do_experiment_given_number_of_times(self):
        percentages = np.zeros(self.random_trials)
        tokenised_sentence_corpus = copy.deepcopy(self.tokenised_sentence_corpus)

        for i in range(self.random_trials):
            tokenised_sentence_corpus = self.do_random_word_subsampling(tokenised_sentence_corpus)

            words_embeddings = self.generate_word_embeddings(tokenised_sentence_corpus)

            result = self.get_clustering_results(words_embeddings)

            percentage = self.get_accuracy(result)
            percentages[i] = percentage

            tokenised_sentence_corpus = copy.deepcopy(self.tokenised_sentence_corpus)

        return percentages


    def do_random_word_subsampling(self, tokenised_sentence_corpus):
        for word, reversed_word in zip(self.most_common_words, self.reversed_words):
            indices = [(i, j) for i, sent in enumerate(tokenised_sentence_corpus) for j, token in enumerate(sent) if token == word]
            random.shuffle(indices)

            # get half of word occurrences which will be replaced by the pseudo word
            number_of_indices = len(indices)//2

            # second half will be replaced by the reversed word
            replace_by_pseudo_words = indices[number_of_indices:]

            for (x, y) in replace_by_pseudo_words:
                tokenised_sentence_corpus[x][y] = reversed_word

        return tokenised_sentence_corpus


    def generate_word_embeddings(self, tokenised_sentence_corpus):
        model = Word2Vec(sentences=tokenised_sentence_corpus, size=self.d, window=self.window, min_count=self.min_count, iter=self.iter, alpha=self.alpha, ns_exponent=0.75)
        # model = FastText(sentences=tokenised_sentence_corpus, size=d, window=5, min_count=5, workers=20, iter=100, ns_exponent=0.75)

        words_embeddings = np.zeros([self.N, self.d])

        for i, word in enumerate(self.all_words):
            words_embeddings[i] = model.wv[word]
        
        return words_embeddings


    def get_clustering_results(self, words_embeddings):
        # clusterer = nltk.cluster.GAAClusterer(N//2)
        # result = clusterer.cluster(words_embeddings, assign_clusters=True)

        # clusterer = KMeans(n_clusters=N//2)
        clusterer = AgglomerativeClustering(n_clusters=NUMBER_OF_WORDS, linkage="ward", affinity="euclidean")
        # clusterer = AgglomerativeClustering(n_clusters=N//2, linkage="complete", affinity="cosine")
        # clusterer = AgglomerativeClustering(n_clusters=N//2, linkage="average", affinity="cosine")
        # clusterer = Birch(n_clusters=N//2)
        
        result = clusterer.fit_predict(words_embeddings)

        # check if there are none empty clusters!
        # calculate rand score? 
        # https://scikit-learn.org/stable/modules/clustering.html#birch
        # print(result)
        return result


    def get_accuracy(self, result):
        matches = 0
        for i in range(0, self.N, 2):
            if result[i] == result[i+1]:
                matches += 1

        percentage = matches/NUMBER_OF_WORDS
        return percentage


In [239]:
random_trials = 10
d = 200
N = NUMBER_OF_WORDS*2
window = 4
min_count = 2
iter = 100
alpha = 0.02

start = time.time()
experiment = DistributionalSemanticsExperiment(random_trials=random_trials, tokenised_sentence_corpus=copy.deepcopy(tokenised_sentence_corpus), \
                                               all_words=all_words, most_common_words=most_common_words, reversed_words=reversed_words, \
                                               N=N, d=d, window=window, min_count=min_count, iter=iter, alpha=alpha)

mean, std = experiment.get_results()
print(time.time() - start)
mean, std

74.52502179145813


(0.9279999999999999, 0.018330302779823338)

In [233]:
# model.wv.most_similar("good")

In [234]:
# print(model)
# print(model.wv.vocab)

In [235]:
# Hyperparameter Selection

# For results create a dictionary of items where the key is the tuple of all parameters and the value is the percentage received
results = dict()
random_trials = 4
N = NUMBER_OF_WORDS*2


def hyperparameter_selection(d_values, window_sizes, min_counts, iter_numbers, alpha_values):
    for d in d_values:
        for window in window_sizes:
            for min_count in min_counts:
                for iter in iter_numbers:
                    for alpha in alpha_values:
                        # parameters = (d, window, min_count, iter, alpha)
                        parameters = f"d={d}, window={window}, min_count={min_count}, iter={iter}, alpha={alpha}"

                        experiment = DistributionalSemanticsExperiment(random_trials=random_trials, tokenised_sentence_corpus=copy.deepcopy(tokenised_sentence_corpus), \
                                                                    all_words=all_words, most_common_words=most_common_words, reversed_words=reversed_words, \
                                                                    N=N, d=d, window=window, min_count=min_count, iter=iter, alpha=alpha)

                        result = experiment.get_results()

                        results[parameters] = result

    return results


# 17280 unique combinations - would take about 216 hours...
# d_values = [50, 100, 150, 200, 250, 300]
# 100 was not working well
d_values = [100, 200, 300]

# window_sizes = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
window_sizes = [3, 4, 5, 6]

# min_counts = [1, 2, 3, 4, 5, 6, 7, 8]
min_counts = [2, 3, 4]

# 200 was too much for my model
# iter_numbers = [50, 100, 150, 200, 250, 300]
iter_numbers = [100, 150]

# alpha_values = [0.001, 0.005, 0.01, 0.025, 0.05, 0.1]
alpha_values = [0.02, 0.025]

# ns_exponent_values = [-1, -0.5, 0, 0.5, 0.75, 1]
# hs_values = [0, 1]
# sg_values = [0, 1] 

start = time.time()
results = hyperparameter_selection(d_values, window_sizes, min_counts, iter_numbers, alpha_values)
print(time.time() - start)

results, len(results)

4876.3112506866455


({'d=100, window=3, min_count=2, iter=100, alpha=0.02': (0.9249999999999999,
   0.02958039891549806),
  'd=100, window=3, min_count=2, iter=100, alpha=0.025': (0.91,
   0.01732050807568879),
  'd=100, window=3, min_count=2, iter=150, alpha=0.02': (0.905,
   0.016583123951777013),
  'd=100, window=3, min_count=2, iter=150, alpha=0.025': (0.8300000000000001,
   0.03),
  'd=100, window=3, min_count=3, iter=100, alpha=0.02': (0.915,
   0.01658312395177697),
  'd=100, window=3, min_count=3, iter=100, alpha=0.025': (0.88,
   0.014142135623730963),
  'd=100, window=3, min_count=3, iter=150, alpha=0.02': (0.8899999999999999,
   0.02999999999999998),
  'd=100, window=3, min_count=3, iter=150, alpha=0.025': (0.86,
   0.020000000000000018),
  'd=100, window=3, min_count=4, iter=100, alpha=0.02': (0.8999999999999999,
   0.028284271247461888),
  'd=100, window=3, min_count=4, iter=100, alpha=0.025': (0.895,
   0.016583123951777013),
  'd=100, window=3, min_count=4, iter=150, alpha=0.02': (0.855,
  

In [236]:
max_value = max(results.values())
result = [(params, value) for params, value in results.items() if value == max_value]
print(f"Maximum achieved {result[0][1]} by using parameters {result[0][0]}")

Maximum achieved (0.945, 0.029580398915498064) by using parameters d=200, window=4, min_count=2, iter=100, alpha=0.02


In [237]:
sorted_results = dict(sorted(results.items(), key=lambda item: item[1][0]))
sorted_results

{'d=100, window=4, min_count=4, iter=150, alpha=0.025': (0.825,
  0.029580398915498064),
 'd=100, window=6, min_count=3, iter=150, alpha=0.025': (0.825,
  0.021794494717703356),
 'd=100, window=3, min_count=2, iter=150, alpha=0.025': (0.8300000000000001,
  0.03),
 'd=100, window=5, min_count=4, iter=150, alpha=0.025': (0.8350000000000001,
  0.03570714214271423),
 'd=100, window=5, min_count=3, iter=150, alpha=0.025': (0.84,
  0.028284271247461888),
 'd=300, window=4, min_count=4, iter=150, alpha=0.025': (0.84,
  0.028284271247461888),
 'd=200, window=4, min_count=4, iter=150, alpha=0.025': (0.845,
  0.008660254037844395),
 'd=100, window=6, min_count=4, iter=150, alpha=0.02': (0.85,
  0.02999999999999998),
 'd=200, window=5, min_count=4, iter=150, alpha=0.025': (0.85,
  0.01732050807568879),
 'd=200, window=6, min_count=4, iter=150, alpha=0.025': (0.85,
  0.010000000000000009),
 'd=300, window=5, min_count=3, iter=150, alpha=0.025': (0.85,
  0.010000000000000009),
 'd=300, window=6, mi